In [1]:
import os
import pandas as pd
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XlImage
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side
from PIL import Image

os.chdir("/scratch/jq2uw/derm_vlms")

In [2]:
DESCRIBE_PROMPT = "Describe the lesion in detail."
CLASSIFY_PROMPT = "Is the lesion malignant or benign, or other?"

VLMS = {
    "DermatoLlama": "results/dermato_llama_predictions_paired.csv",
    "LLaVA-Derm": "results/llava_derm_predictions_paired.csv",
    "SkinGPT-4": "results/skingpt4_predictions_paired.csv",
}

IMAGE_DIR = "results/images"

RESPONSE_COLUMNS = [
    f"describe: {DESCRIBE_PROMPT}",
    f"classify: {CLASSIFY_PROMPT}",
    "describe_then_classify",
]

COLUMN_RENAME = {
    "describe": RESPONSE_COLUMNS[0],
    "classify": RESPONSE_COLUMNS[1],
}

SURVEY_COLUMNS = [
    "survey question 1",
    "survey question 2",
    "survey question 3",
]

IMG_HEIGHT_PX = 150
ROW_HEIGHT_PTS = 120

In [ ]:
wb = Workbook()
wb.remove(wb.active)

HEADERS = ["id", "image"] + RESPONSE_COLUMNS + SURVEY_COLUMNS

header_font = Font(bold=True, size=11)
header_fill = PatternFill(start_color="D9E1F2", end_color="D9E1F2", fill_type="solid")
thin_border = Border(
    left=Side(style="thin"), right=Side(style="thin"),
    top=Side(style="thin"), bottom=Side(style="thin"),
)
wrap_top = Alignment(wrap_text=True, vertical="top")
center_v = Alignment(vertical="center")
center_both = Alignment(vertical="center", horizontal="center")

for vlm_name, csv_path in VLMS.items():
    df = pd.read_csv(csv_path).rename(columns=COLUMN_RENAME)
    ws = wb.create_sheet(title=vlm_name)

    # --- headers ---
    for col_idx, header in enumerate(HEADERS, 1):
        cell = ws.cell(row=1, column=col_idx, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
        cell.border = thin_border
    ws.row_dimensions[1].height = 40

    # --- column widths ---
    ws.column_dimensions["A"].width = 15       # id
    ws.column_dimensions["B"].width = 28       # image
    for i, _ in enumerate(RESPONSE_COLUMNS):   # response cols
        ws.column_dimensions[get_column_letter(3 + i)].width = 55
    for i, _ in enumerate(SURVEY_COLUMNS):     # survey cols
        ws.column_dimensions[get_column_letter(3 + len(RESPONSE_COLUMNS) + i)].width = 22

    # --- data rows ---
    for row_idx, (_, row) in enumerate(df.iterrows(), 2):
        # id
        c = ws.cell(row=row_idx, column=1, value=row["id"])
        c.alignment = center_v
        c.border = thin_border

        # embedded image
        img_path = os.path.join(IMAGE_DIR, f"{row['id']}.jpg")
        if os.path.exists(img_path):
            pil_img = Image.open(img_path)
            aspect = pil_img.width / pil_img.height
            xl_img = XlImage(img_path)
            xl_img.height = IMG_HEIGHT_PX
            xl_img.width = int(IMG_HEIGHT_PX * aspect)
            ws.add_image(xl_img, f"B{row_idx}")
        ws.cell(row=row_idx, column=2).border = thin_border

        # response columns
        for col_off, col_name in enumerate(RESPONSE_COLUMNS):
            c = ws.cell(row=row_idx, column=3 + col_off, value=row[col_name])
            c.alignment = wrap_top
            c.border = thin_border

        # empty survey columns
        for col_off in range(len(SURVEY_COLUMNS)):
            c = ws.cell(row=row_idx, column=3 + len(RESPONSE_COLUMNS) + col_off)
            c.border = thin_border
            c.alignment = center_both

        ws.row_dimensions[row_idx].height = ROW_HEIGHT_PTS

    ws.freeze_panes = "A2"
    print(f"  ✓ {vlm_name}: {len(df)} rows")

output_path = "results/interface1.xlsx"
wb.save(output_path)
print(f"\nSaved → {output_path}")

  ✓ DermatoLlama: 30 rows
  ✓ LLaVA-Derm: 30 rows
  ✓ SkinGPT-4: 30 rows

Saved → res_eng/interface1.xlsx
